## Neural Networks with PSD features, theta and alpha bands only


In [1]:
#Import necessary libraries

import pandas as pd
import numpy as np
from scipy import signal
import os
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pickle as pkl
import itertools 
import glob
from sklearn import svm 
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix, f1_score
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

%matplotlib inline 
# %matplotlib qt


In [2]:
# Import datasets
psd_df = pd.read_pickle("F:\EEG-data\\think-read\\featureExtraction\PSD_only\PSD_df_12.pkl")
auc_df = pd.read_pickle("F:\EEG-data\\think-read\\featureExtraction\AUC_only\AUC_df_12.pkl")
AP_df = pd.read_pickle("F:\EEG-data\\think-read\\featureExtraction\AUC_PSD\AUC_PSD_df_12.pkl")

In [3]:
#Baseline model
def baseline_func(no_features):
    ANN_model = keras.Sequential([
    # keras.layers.Flatten(input_shape=(8,32)),
    keras.layers.Dense(no_features,input_dim=no_features,activation='relu'),
    keras.layers.Dense(no_features/2, activation='relu'),
    keras.layers.Dense(no_features/4, activation='relu'),
    keras.layers.Dense(1,activation='sigmoid')])

    ANN_model.compile(optimizer="adam",
                loss="binary_crossentropy",
                metrics = ['accuracy'])
    return ANN_model

#Cross Validation and Prediction
def apply_CV_predict(baseline_func,df,filename,savedir):
    #Split into training and test sets 
    class_names = ["T","R"]
    X = df.iloc[:,:-1].values.astype(np.float32)
    y = df['Action'].apply(class_names.index).values.astype(np.float32)
    sss = StratifiedShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
    for train_index, test_index in sss.split(X,y):
                X_train, X_test = X[train_index],X[test_index]
                y_train, y_test = y[train_index],y[test_index]
    no_features = X.shape[1]
    #Cross Validation 
    epochsList = list(range(1,20))
    mean_results = [] 
    kfold = StratifiedShuffleSplit(n_splits=10)
    for epoch in epochsList:
        estimator=KerasClassifier(build_fn=lambda:baseline_func(no_features),epochs=epoch,batch_size=5,verbose=0)
        result = np.mean(cross_val_score(estimator,X_train,y_train,cv=kfold,scoring="f1_macro"))
        mean_results.append(result)
    mean_results = np.array(mean_results)
    best_epoch = epochsList[np.argmax(mean_results)]
    #Test set evaluation
    final_estimator = KerasClassifier(build_fn=lambda:baseline_func(no_features),epochs=best_epoch,batch_size=5,verbose=0)
    final_estimator.fit(X_train,y_train)
    y_pred = final_estimator.predict(X_test)
    #F1 Score
    f1Score = f1_score(y_test,y_pred)
    #Confusion matrix
    cm = confusion_matrix(y_test,y_pred,normalize='true')
    # fig,ax = plt.subplots()
    # ax.set_title(filename+'CM')
    # plot_confusion_matrix(final_estimator,X_test,y_test,labels=class_names,ax=ax,normalize='true')
    # plt.savefig(savedir+filename+'.png')
    # plt.close()
    return f1Score,cm

In [4]:
#Directories to save to
savedir = "F:\EEG-data\\think-read\ANN/"
PSD_filename = "PSD_"
AUC_filename = "AUC_"
AP_filename = "AP_"

PSD_f1Score,PSD_cm = apply_CV_predict(baseline_func,psd_df,PSD_filename,savedir)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [5]:
AUC_f1Score,AUC_cm = apply_CV_predict(baseline_func,auc_df,AUC_filename,savedir)


In [6]:
AP_f1Score,AP_cm = apply_CV_predict(baseline_func,AP_df,AP_filename,savedir)

In [7]:
print("PSD F1 Score:",PSD_f1Score)
print("AUC F1 Score:",AUC_f1Score)
print("AP F1 Score:",AP_f1Score)

PSD F1 Score: 0.8721804511278196
AUC F1 Score: 0.8175182481751825
AP F1 Score: 0.8656716417910447


In [8]:
print(PSD_cm)
print(AUC_cm)
print(AP_cm)

[[0.828125 0.171875]
 [0.09375  0.90625 ]]
[[0.734375 0.265625]
 [0.125    0.875   ]]
[[0.8125  0.1875 ]
 [0.09375 0.90625]]
